In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv("final_df.csv")

In [5]:
df.head()

,Category,Resume,processed
0,UNK,Product Developer Product Developer Product Ma...,product developer product developer product ma...
1,UNK,"Network Administrator Network <span class=""hl""...",network administrator network span class admin...
2,UNK,"Systems Administrator <span class=""hl"">Systems...",systems administrator span class systems span ...
3,UNK,Sr.Fullstack Developer Sr.Fullstack <span clas...,fullstack developer fullstack span class devel...
4,UNK,Identity and Access Management Project Manager...,identity management manager identity managemen...


In [40]:
df=df.tail(20000)

In [17]:
df1=pd.read_csv('worldcities.csv')

In [18]:
df1=df1[(df1['country']=='India') | (df1['country'] == 'United States')]

In [19]:
cities=df1['city'].tolist()

In [20]:
for i in range(0,len(cities)):
  cities[i]=cities[i].lower()

In [36]:
df=df.dropna()

In [41]:
df.shape

(20000, 3)

In [70]:
!pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.utils import simple_preprocess
import re

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
# List of months in long format
# List of months in long format with all alphabets in lowercase
months_long= [
    "january", "february", "march", "april", "may", "june",
    "july", "august", "september", "october", "november", "december"
]

# List of months in short format with all alphabets in lowercase
months_short = [
    "jan", "feb", "mar", "apr", "may", "jun",
    "jul", "aug", "sep", "oct", "nov", "dec"
]
# Extend the stopwords list with your custom deletion words
del_words = ['name', 'city', 'state', 'country', 'fullname', 'company', 'resume','intro', 'curriculum', 'vitae', 'address', 'phone',
             'email', 'linkedin', 'profile', 'summary', 'objective', 'experience', 'education', 'skill', 'skills','bachelor',
             'reference', 'references', 'contact', 'detail', 'details', 'mail', 'gmail', 'yahoo', 'hotmail', 'mailing',
             'twitter', 'facebook', 'instagram','intro','using', 'website', 'web', 'url', 'www', 'year', 'month','months','requirement','first', 'last', 'xxxx', 'rstlast', 'rstlast', 'github', 'rstlast', 'university', 'expected', 'bachelor', 'science','project', 'description', 'responsibility', 'role','time','nagpur', 'secondary','exprience']

stop_words = stopwords.words('english') + del_words +cities+months_long+months_short
# ct=0
def preprocess(text):
    text = re.sub('http\S+\s*', ' ', text)  # remove URLs
    text = re.sub('RT|cc', ' ', text)  # remove RT and cc
    text = re.sub('#\S+', '', text)  # remove hashtags
    text = re.sub('@\S+', '  ', text)  # remove mentions
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', text)  # remove punctuations
    text = re.sub(r'[^\x00-\x7f]',r' ', text)  # remove non-ASCII characters
    text = re.sub('\s+', ' ', text)  # remove extra whitespace
    text = re.sub('\d+', '', text)  # remove numbers
    text = text.lower()  # convert to lowercase
    result = []
    for token in simple_preprocess(text):
        if token not in stop_words and len(token) > 3:
              result.append(token)
    return " ".join(result)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
from tqdm import tqdm

# Assuming df is your DataFrame and 'Resume' is the column you're processing
tqdm.pandas()  # Enable progress_apply for pandas

# Apply your preprocess function with progress bar
df['processed'] = df['Resume'].progress_map(preprocess)

In [42]:
texts = df['processed'].tolist()
# train_texts, val_texts = train_test_split(texts, test_size=0.1)

In [43]:
len(texts)

20000

In [15]:
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
import torch
from torch.utils.data import Dataset

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize the model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

2024-03-27 10:59:08.489603: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 10:59:08.489782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 10:59:08.681216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer):
        # Tokenize all texts at once, not one by one
        self.encodings = tokenizer(texts, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

    def __len__(self):
        # The length of the dataset is the number of items in 'input_ids'
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

# Initialize your custom dataset
dataset = CustomDataset(texts, tokenizer)

# Data collator that will dynamically mask tokens
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)


In [47]:
data_collator

DataCollatorForLanguageModeling(tokenizer=BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, mlm=True, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False,

In [48]:
output_dir="BERT_finetuned_MLM"
# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained(output_dir)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,4.201200
1000,3.754300
1500,3.599800
2000,3.466900
2500,3.435300
3000,3.343000
3500,3.310200
4000,3.267200
4500,3.224000
5000,3.200500
